<a href="https://colab.research.google.com/github/c-marq/cap4767-data-mining/blob/main/demos/week02_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 2 Demo — Time Series Forecasting: SARIMAX + Prophet
**CAP4767 Data Mining with Python** | Miami Dade College — Kendall Campus

---

**What we're building today:** A complete end-to-end forecasting pipeline using two industry-standard models — SARIMAX (classical statistics) and Prophet (Facebook/Meta's modern approach) — then comparing them head-to-head.

**Why this matters for data mining:** Forecasting is how businesses turn historical patterns into forward-looking decisions. Hotels predict occupancy to set room prices. Retailers forecast demand to manage inventory. Public health agencies forecast disease spread to allocate resources. The techniques you learn today are the same ones powering those systems.

**Dataset:** Australian Tourism — quarterly international holiday trips (1998–2016, 76 quarters). We use this as a "comparable tourism economy" exercise — the seasonal patterns mirror what we see in South Florida.

**Key concept:** We're comparing two fundamentally different philosophies:
- **SARIMAX:** "Model the math of the pattern" — explicitly defines trend, seasonality, and autocorrelation with parameters (p,d,q)(P,D,Q,s)
- **Prophet:** "Fit a flexible curve" — decomposes time series into trend + seasonality + holidays using an additive model

**Adapted from:** Adhwaith Shankara Narayanan, *End to End Time Series Forecasting with SARIMAX and Prophet*

<div style="background-color: #D6EAF8; border-left: 5px solid #2E86C1; padding: 15px; margin: 15px 0; border-radius: 4px;">
<strong style="color: #1A5276;">💡 WHY ARE WE DOING THIS?</strong><br>
<strong>Where does this fit in the data mining pipeline?</strong><br><br>Week 1 gave us the toolkit — resampling, rolling windows, datetime indexing. Today we plug those skills into actual predictive models. This completes <strong>Competency 3</strong> (Forecasting) entirely. The temporal thinking you build here also feeds into Week 4 (churn prediction uses time-based features) and Week 7 (RFM recency is a time-series concept).
</div>

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
<strong style="color: #1E8449;">✅ DO THIS</strong><br>
Run the next two cells to install dependencies and load libraries. <strong>Do not modify.</strong><br><br>The first cell installs <code>pmdarima</code> (for auto_arima) and <code>prophet</code>. This takes about 30–60 seconds in Colab.
</div>

In [ ]:
# ============================================================
# Install dependencies — Run this cell. Do not modify.
# ============================================================
!pip install -q pmdarima prophet

print("✅ pmdarima and prophet installed successfully.")

In [ ]:
# ============================================================
# Imports — Run this cell. Do not modify.
# ============================================================
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima import auto_arima
from prophet import Prophet
from sklearn.metrics import mean_squared_error, r2_score

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import logging

# Suppress verbose Prophet/cmdstanpy output
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)
warnings.filterwarnings("ignore")

print("✅ Libraries loaded successfully.")

---
## Section 1: Data Loading & Preprocessing

<div style="background-color: #D6EAF8; border-left: 5px solid #2E86C1; padding: 15px; margin: 15px 0; border-radius: 4px;">
<strong style="color: #1A5276;">💡 WHY ARE WE DOING THIS?</strong><br>
The raw tourism dataset has 23,000+ rows — one per region, per quarter, per trip purpose. We need to filter for Holiday trips only, aggregate all regions into a national total, and set the quarterly datetime index. This is standard time-series preprocessing: go from granular transactional data to a single aggregated series.
</div>

In [ ]:
# ============================================================
# Load and preprocess — Run this cell. Do not modify.
# ============================================================
tourism_url = "https://raw.githubusercontent.com/c-marq/cap4767-data-mining/main/data/tourism.csv"
tourism_df = pd.read_csv(tourism_url, usecols=["Quarter", "Purpose", "Trips"])

print(f"Raw dataset: {tourism_df.shape[0]:,} rows, {tourism_df.shape[1]} columns")
print(f"Trip purposes: {tourism_df['Purpose'].unique().tolist()}")
print(f"Quarter range: {tourism_df['Quarter'].min()} to {tourism_df['Quarter'].max()}")
tourism_df.head()

In [ ]:
def preprocess_tourism_data(df):
    """
    Filter for Holiday trips, aggregate to quarterly national totals,
    and set datetime index with quarterly frequency.
    """
    # Keep only Holiday trips
    df = df[df["Purpose"] == "Holiday"].copy()

    # Convert quarter strings to datetime
    df["Quarter"] = pd.to_datetime(df["Quarter"])

    # Convert trips to integer (removes fractional survey artifacts)
    df["Trips"] = df["Trips"].astype(np.int64)

    # Sort by quarter
    df.sort_values("Quarter", inplace=True)

    # Aggregate: sum all regional trips per quarter into one national number
    aggregated = (
        df.groupby("Quarter")["Trips"]
        .sum()
        .rename("Trips")
    )

    # Set quarterly frequency so time-series methods work correctly
    aggregated = aggregated.asfreq("QS")

    return aggregated

# Run preprocessing
ts_data = preprocess_tourism_data(tourism_df)
print(f"\n✅ Aggregated time series: {len(ts_data)} quarters")
print(f"   Date range: {ts_data.index.min().strftime('%Y Q%q')} to {ts_data.index.max().strftime('%Y Q%q')}")
print(f"   Mean quarterly trips: {ts_data.mean():,.0f}")
ts_data.head()

<div style="background-color: #FADBD8; border-left: 5px solid #E74C3C; padding: 15px; margin: 15px 0; border-radius: 4px;">
<strong style="color: #922B21;">🛑 STOP AND CHECK</strong><br>
<strong>Checkpoint — Section 1</strong><br><br>You should see:<br>• Raw dataset: 23,408 rows, 3 columns<br>• Aggregated time series: <strong>76 quarters</strong> (1998 Q1 to 2016 Q4)<br>• Mean quarterly trips: ~10,000–11,000<br>• The index should be a <code>DatetimeIndex</code> with <code>freq='QS-JAN'</code>
</div>

---
## Section 2: Exploratory Data Analysis

<div style="background-color: #D6EAF8; border-left: 5px solid #2E86C1; padding: 15px; margin: 15px 0; border-radius: 4px;">
<strong style="color: #1A5276;">💡 WHY ARE WE DOING THIS?</strong><br>
Before modeling, we need to answer three questions:<br><br>1. <strong>What does the series look like?</strong> — Plot it, check for obvious trend and seasonality<br>2. <strong>Is it stationary?</strong> — The ADF (Augmented Dickey-Fuller) test tells us. SARIMAX needs to know this to set the <code>d</code> parameter.<br>3. <strong>What are the components?</strong> — Seasonal decomposition separates trend, seasonality, and residual noise.<br><br>This is the same workflow from Chapter 1, now applied with purpose — each answer feeds directly into model configuration.
</div>

### Time Series Plot

In [ ]:
# Visualize the full series
plt.figure(figsize=(12, 5))
plt.plot(ts_data.index, ts_data.values, color='steelblue', linewidth=1.5)
plt.title("Australian Holiday Trips — Quarterly (1998–2016)", fontsize=14)
plt.xlabel("Year")
plt.ylabel("Total Trips (thousands)")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Quick stats
print(ts_data.describe().round(0))

### Augmented Dickey-Fuller (ADF) Test for Stationarity

In [ ]:
# ADF Test: Is the series stationary?
adf_result = adfuller(ts_data.dropna(), autolag="AIC")

print("Augmented Dickey-Fuller Test Results")
print("=" * 40)
print(f"Test Statistic : {adf_result[0]:.4f}")
print(f"P-value        : {adf_result[1]:.4f}")
print(f"Lags Used      : {adf_result[2]}")
print(f"Observations   : {adf_result[3]}")
print()

if adf_result[1] < 0.05:
    print("✅ P-value < 0.05 → Data IS stationary")
    print("   → SARIMAX may use d=0 (no differencing needed)")
else:
    print("⚠️  P-value >= 0.05 → Data is NOT stationary")
    print("   → SARIMAX will likely need d=1 (first differencing)")

<div style="background-color: #FEF9E7; border-left: 5px solid #F1C40F; padding: 15px; margin: 15px 0; border-radius: 4px;">
<strong style="color: #7D6608;">⚠️ COMMON MISTAKE</strong><br>
<strong>What does stationarity mean in plain English?</strong><br><br>A stationary series has a <em>consistent average</em> over time — it doesn't trend up or down permanently. Think of ocean waves: the height varies, but the average sea level stays the same. If sea level is <em>rising</em> (like global warming), that's non-stationary.<br><br>SARIMAX needs stationary data to work properly. If the data isn't stationary, the <code>d</code> parameter tells it to <em>difference</em> the data first (subtract each value from the previous one) to remove the trend.
</div>

### Seasonal Decomposition

In [ ]:
# Decompose into Trend + Seasonal + Residual
decomposition = seasonal_decompose(ts_data, model='additive')

fig = decomposition.plot()
fig.set_size_inches(12, 8)
fig.suptitle("Seasonal Decomposition — Australian Holiday Trips", fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

print("What to look for:")
print("• Trend panel:  Is there an overall upward or downward drift?")
print("• Seasonal panel: Do the peaks repeat at the same interval? (Should be every 4 quarters)")
print("• Residual panel: Is it random noise, or are there patterns the model isn't capturing?")

<div style="background-color: #FADBD8; border-left: 5px solid #E74C3C; padding: 15px; margin: 15px 0; border-radius: 4px;">
<strong style="color: #922B21;">🛑 STOP AND CHECK</strong><br>
<strong>Checkpoint — Section 2</strong><br><br>Your EDA should reveal:<br>• <strong>Time series plot:</strong> Clear repeating peaks (Q1 = summer in Australia = peak holiday season) with a gradual upward trend<br>• <strong>ADF test:</strong> The p-value will determine stationarity — check whether it's above or below 0.05<br>• <strong>Decomposition:</strong> The seasonal component should show a regular 4-quarter cycle. The trend should show gradual growth.
</div>

---
## Section 3: Train/Test Split

<div style="background-color: #D6EAF8; border-left: 5px solid #2E86C1; padding: 15px; margin: 15px 0; border-radius: 4px;">
<strong style="color: #1A5276;">💡 WHY ARE WE DOING THIS?</strong><br>
Time series splitting is <strong>different from regular ML</strong>. We can't randomly shuffle — the order matters. We always train on the <em>past</em> and test on the <em>future</em>. This is called a <strong>temporal split</strong>.<br><br>We'll use the first 64 quarters for training (1998–2013) and the last 12 quarters for testing (2014–2016). This gives us 3 full years of out-of-sample data to evaluate our forecasts.
</div>

In [ ]:
def train_test_split_ts(series, train_size):
    """
    Temporal split — train on the past, test on the future.
    Never shuffle time series data!
    """
    train = series.iloc[:train_size]
    test = series.iloc[train_size:]
    return train, test

# Split: 64 quarters train / 12 quarters test
train_data, test_data = train_test_split_ts(ts_data, train_size=64)

print(f"Train: {len(train_data)} quarters ({train_data.index.min().year}–{train_data.index.max().year})")
print(f"Test:  {len(test_data)} quarters ({test_data.index.min().year}–{test_data.index.max().year})")

# Visualize the split
plt.figure(figsize=(12, 5))
plt.plot(train_data.index, train_data, label=f"Train ({len(train_data)} Q)", color="steelblue")
plt.plot(test_data.index, test_data, label=f"Test ({len(test_data)} Q)", color="darkorange")
plt.axvline(x=test_data.index[0], color='red', linestyle='--', alpha=0.7, label='Split Point')
plt.title("Temporal Train/Test Split")
plt.xlabel("Year")
plt.ylabel("Trips")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

<div style="background-color: #FEF9E7; border-left: 5px solid #F1C40F; padding: 15px; margin: 15px 0; border-radius: 4px;">
<strong style="color: #7D6608;">⚠️ COMMON MISTAKE</strong><br>
<strong>Why 64/12 and not 80/20?</strong><br><br>With only 76 quarters, we want the test set to cover at least 3 full seasonal cycles (12 quarters = 3 years). This lets us evaluate whether the model captures seasonality — not just the trend. If we only tested on 1 year, a model that gets the trend right but misses the seasonal peaks would look good by accident.
</div>

---
## Section 4: SARIMAX — Parameter Selection with auto_arima

<div style="background-color: #D6EAF8; border-left: 5px solid #2E86C1; padding: 15px; margin: 15px 0; border-radius: 4px;">
<strong style="color: #1A5276;">💡 WHY ARE WE DOING THIS?</strong><br>
SARIMAX has 7 parameters: <code>(p, d, q)(P, D, Q, s)</code>. Choosing them manually requires deep statistical expertise — reading ACF/PACF plots, testing multiple combinations, checking AIC scores. That's a graduate-level skill.<br><br><code>auto_arima</code> from the <code>pmdarima</code> library automates this search. It tries thousands of parameter combinations and picks the one with the best AIC (Akaike Information Criterion — a measure of model fit that penalizes complexity). Think of it as a smart search algorithm that does in 30 seconds what would take a statistician hours.
</div>

### What do the SARIMAX parameters mean?

| Parameter | Name | What it controls |
|-----------|------|------------------|
| `p` | AR order | How many past values influence the current value |
| `d` | Differencing | How many times to subtract consecutive values (removes trend) |
| `q` | MA order | How many past forecast errors influence the current value |
| `P` | Seasonal AR | Same as `p`, but for the seasonal pattern |
| `D` | Seasonal differencing | Removes seasonal trend |
| `Q` | Seasonal MA | Same as `q`, but for seasonal errors |
| `s` | Seasonal period | Length of one seasonal cycle (4 for quarterly data) |

In [ ]:
def run_auto_arima(train_data, seasonal=True, m=4):
    """
    Use auto_arima to find the best SARIMAX parameters.
    m=4 because our data is quarterly (4 quarters = 1 seasonal cycle).
    """
    model = auto_arima(
        train_data,
        seasonal=seasonal,
        m=m,
        suppress_warnings=True,
        error_action="ignore",
        trace=False
    )

    order = model.order
    seasonal_order = model.seasonal_order

    params = {
        "p": order[0], "d": order[1], "q": order[2],
        "P": seasonal_order[0], "D": seasonal_order[1],
        "Q": seasonal_order[2], "s": seasonal_order[3]
    }
    return model, params

# Run auto_arima — this takes ~15-30 seconds
print("Running auto_arima parameter search...")
auto_model, sarimax_params = run_auto_arima(train_data, seasonal=True, m=4)

print(f"\n✅ Best SARIMAX parameters found:")
print(f"   Order:          ({sarimax_params['p']}, {sarimax_params['d']}, {sarimax_params['q']})")
print(f"   Seasonal Order: ({sarimax_params['P']}, {sarimax_params['D']}, {sarimax_params['Q']}, {sarimax_params['s']})")
print(f"   AIC:            {auto_model.aic():.2f}")
print(f"\n{auto_model.summary()}")

<div style="background-color: #FADBD8; border-left: 5px solid #E74C3C; padding: 15px; margin: 15px 0; border-radius: 4px;">
<strong style="color: #922B21;">🛑 STOP AND CHECK</strong><br>
<strong>Checkpoint — Section 4</strong><br><br>auto_arima should finish in under 30 seconds and print a parameter summary. You should see values like <code>(p, d, q) = (0, 1, 1)</code> and <code>(P, D, Q, 4) = (0, 1, 1, 4)</code> — though exact values may vary slightly. The <code>s=4</code> confirms it detected quarterly seasonality.<br><br>If you get a convergence warning, that's normal — auto_arima tries many combinations and some don't converge.
</div>

---
## Section 5: SARIMAX — Fit, Forecast, and Evaluate

<div style="background-color: #D6EAF8; border-left: 5px solid #2E86C1; padding: 15px; margin: 15px 0; border-radius: 4px;">
<strong style="color: #1A5276;">💡 WHY ARE WE DOING THIS?</strong><br>
Now we take the parameters auto_arima found and build the actual SARIMAX model. The workflow is: <strong>fit on training data → forecast the test period → compare forecast to actuals → measure error</strong>. We use two metrics:<br><br>• <strong>RMSE</strong> (Root Mean Squared Error) — average forecast error in the same units as the data (trips). Lower is better.<br>• <strong>R²</strong> (R-squared) — proportion of variance explained. 1.0 = perfect, 0.0 = no better than guessing the mean.
</div>

In [ ]:
def sarimax_model(train, test=None, params=None, forecast_periods=None):
    """
    Fit SARIMAX model and optionally evaluate against test data.
    Returns forecast (and metrics if test data provided).
    """
    model = SARIMAX(
        train,
        order=(params["p"], params["d"], params["q"]),
        seasonal_order=(params["P"], params["D"], params["Q"], params["s"]),
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    result = model.fit(disp=False)

    # Forecast length
    steps = len(test) if test is not None else forecast_periods
    forecast = result.forecast(steps=steps)

    # Evaluate if test data provided
    if test is not None:
        rmse = np.sqrt(mean_squared_error(test, forecast))
        r2 = r2_score(test, forecast)
        return forecast, rmse, r2
    else:
        return forecast

# Fit and evaluate on test set
sarimax_forecast, sarimax_rmse, sarimax_r2 = sarimax_model(
    train=train_data,
    test=test_data,
    params=sarimax_params
)

print("SARIMAX Test Set Performance")
print("=" * 40)
print(f"RMSE: {sarimax_rmse:,.0f} trips")
print(f"R²:   {sarimax_r2:.4f}")

### Visualize: SARIMAX Forecast vs Actuals

In [ ]:
def plot_forecast(train, test, forecast, title, future_forecast=None, future_index=None):
    """
    Plot train, test actuals, test forecast, and optional future forecast.
    """
    plt.figure(figsize=(12, 5))
    plt.plot(train.index, train, label="Train", color="steelblue")
    plt.plot(test.index, test, label="Test (Actual)", color="darkorange", linewidth=2)
    plt.plot(test.index, forecast, label="Forecast", linestyle="--", color="green", linewidth=2)

    if future_forecast is not None and future_index is not None:
        plt.plot(future_index, future_forecast, label="Future Forecast",
                 linestyle="--", color="red", linewidth=2)

    plt.title(title, fontsize=14)
    plt.xlabel("Quarter")
    plt.ylabel("Trips")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

# Plot SARIMAX results
plot_forecast(
    train=train_data,
    test=test_data,
    forecast=sarimax_forecast,
    title=f"SARIMAX Forecast vs Actuals (RMSE={sarimax_rmse:,.0f}, R²={sarimax_r2:.3f})"
)

### SARIMAX Future Forecast (Beyond Known Data)

In [ ]:
# Retrain on ALL data, then forecast 8 quarters into the unknown future
full_data = pd.concat([train_data, test_data])
future_periods = 8

sarimax_future = sarimax_model(
    train=full_data,
    params=sarimax_params,
    forecast_periods=future_periods
)

# Build future date index
future_index = pd.date_range(
    start=full_data.index[-1] + pd.tseries.frequencies.to_offset("QS"),
    periods=future_periods,
    freq="QS"
)

# Plot everything: train + test + future
plot_forecast(
    train=train_data,
    test=test_data,
    forecast=sarimax_forecast,
    future_forecast=sarimax_future,
    future_index=future_index,
    title="SARIMAX — Full Pipeline: Historical + Test + Future Forecast"
)

print("\n📊 SARIMAX Future Forecast (8 Quarters):")
print(pd.DataFrame({"Quarter": future_index, "Predicted_Trips": sarimax_future.round().astype(int).values}
    ).to_string(index=False))

<div style="background-color: #FADBD8; border-left: 5px solid #E74C3C; padding: 15px; margin: 15px 0; border-radius: 4px;">
<strong style="color: #922B21;">🛑 STOP AND CHECK</strong><br>
<strong>Checkpoint — Section 5</strong><br><br>You should see three outputs:<br>• <strong>Metrics:</strong> RMSE and R² printed. R² should be positive (better than guessing the mean). If R² is negative, the model is performing <em>worse</em> than the baseline — that's a sign something went wrong.<br>• <strong>Forecast plot:</strong> The green dashed line (forecast) should roughly follow the orange line (actuals) during the test period.<br>• <strong>Future forecast:</strong> The red dashed line extends 8 quarters beyond the known data. It should continue the seasonal pattern.
</div>

---
## Section 6: Prophet — Fit, Forecast, and Evaluate

<div style="background-color: #D6EAF8; border-left: 5px solid #2E86C1; padding: 15px; margin: 15px 0; border-radius: 4px;">
<strong style="color: #1A5276;">💡 WHY ARE WE DOING THIS?</strong><br>
Prophet takes a completely different approach than SARIMAX. Instead of modeling autocorrelation, it fits a <strong>decomposable model</strong>: <code>y(t) = trend(t) + seasonality(t) + holidays(t) + error(t)</code>.<br><br>Key differences:<br>• Prophet doesn't require stationary data — it handles trend internally<br>• It uses a different input format: a DataFrame with columns <code>ds</code> (dates) and <code>y</code> (values)<br>• Parameters control <em>flexibility</em> rather than lag structure — how quickly can the trend change? How strong is the seasonality?<br><br>We'll use reasonable default parameters instead of a grid search. For quarterly tourism data, the defaults below work well.
</div>

### Prophet Parameters (Direct Defaults)

| Parameter | Value | What it controls |
|-----------|-------|------------------|
| `changepoint_prior_scale` | 0.05 | How flexible the trend is (higher = more wiggly) |
| `seasonality_prior_scale` | 10.0 | Strength of seasonal component |
| `holidays_prior_scale` | 0.1 | Influence of holidays (low — our data is quarterly) |
| `seasonality_mode` | `'multiplicative'` | Seasonal effect grows with the trend (vs additive) |
| `changepoint_range` | 0.85 | How far into the training data changepoints are allowed |

In [ ]:
# Prophet parameters — reasonable defaults for quarterly tourism data
prophet_params = {
    "changepoint_prior_scale": 0.05,
    "seasonality_prior_scale": 10.0,
    "holidays_prior_scale": 0.1,
    "seasonality_mode": "multiplicative",
    "changepoint_range": 0.85
}

def prophet_model(train, test=None, params=None, forecast_periods=None):
    """
    Fit Prophet model and optionally evaluate against test data.
    Prophet requires a DataFrame with columns 'ds' (date) and 'y' (value).
    """
    train_df = pd.DataFrame({"ds": train.index, "y": train.values})

    model = Prophet(
        changepoint_prior_scale=params["changepoint_prior_scale"],
        seasonality_prior_scale=params["seasonality_prior_scale"],
        holidays_prior_scale=params["holidays_prior_scale"],
        seasonality_mode=params["seasonality_mode"],
        changepoint_range=params["changepoint_range"],
        yearly_seasonality=True,
        weekly_seasonality=False,   # Not relevant for quarterly data
        daily_seasonality=False     # Not relevant for quarterly data
    )

    model.fit(train_df)

    # Determine forecast horizon
    periods = len(test) if test is not None else forecast_periods

    future = model.make_future_dataframe(periods=periods, freq="QS")
    forecast = model.predict(future)
    yhat = forecast["yhat"].iloc[-periods:].values

    # Evaluate if test data provided
    if test is not None:
        rmse = np.sqrt(mean_squared_error(test.values, yhat))
        r2 = r2_score(test.values, yhat)
        return yhat, rmse, r2, model
    else:
        return yhat, model

# Fit and evaluate on test set
prophet_forecast, prophet_rmse, prophet_r2, fitted_prophet = prophet_model(
    train=train_data,
    test=test_data,
    params=prophet_params
)

print("Prophet Test Set Performance")
print("=" * 40)
print(f"RMSE: {prophet_rmse:,.0f} trips")
print(f"R²:   {prophet_r2:.4f}")

### Prophet's Built-In Component Plots

In [ ]:
# Prophet has its own decomposition — trend + yearly seasonality
fig = fitted_prophet.plot_components(
    fitted_prophet.predict(
        fitted_prophet.make_future_dataframe(periods=len(test_data), freq="QS")
    )
)
plt.suptitle("Prophet Components — Trend + Yearly Seasonality", fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

### Visualize: Prophet Forecast vs Actuals

In [ ]:
# Plot Prophet results
plot_forecast(
    train=train_data,
    test=test_data,
    forecast=prophet_forecast,
    title=f"Prophet Forecast vs Actuals (RMSE={prophet_rmse:,.0f}, R²={prophet_r2:.3f})"
)

### Prophet Future Forecast

In [ ]:
# Retrain Prophet on ALL data, forecast 8 quarters ahead
prophet_future, _ = prophet_model(
    train=full_data,
    params=prophet_params,
    forecast_periods=future_periods
)

# Plot everything
plot_forecast(
    train=train_data,
    test=test_data,
    forecast=prophet_forecast,
    future_forecast=prophet_future,
    future_index=future_index,
    title="Prophet — Full Pipeline: Historical + Test + Future Forecast"
)

print("\n📊 Prophet Future Forecast (8 Quarters):")
print(pd.DataFrame({"Quarter": future_index, "Predicted_Trips": np.round(prophet_future).astype(int)
    }).to_string(index=False))

<div style="background-color: #FADBD8; border-left: 5px solid #E74C3C; padding: 15px; margin: 15px 0; border-radius: 4px;">
<strong style="color: #922B21;">🛑 STOP AND CHECK</strong><br>
<strong>Checkpoint — Section 6</strong><br><br>You should see Prophet's metrics, component plots, and forecast visualization. Compare mentally to SARIMAX:<br>• Which model has the lower RMSE?<br>• Which forecast line tracks the orange actuals more closely?<br>• Do the future forecasts look reasonable, or does either model produce something unrealistic?
</div>

---
## Section 7: Head-to-Head Comparison

<div style="background-color: #D6EAF8; border-left: 5px solid #2E86C1; padding: 15px; margin: 15px 0; border-radius: 4px;">
<strong style="color: #1A5276;">💡 WHY ARE WE DOING THIS?</strong><br>
This is the payoff. We've run two completely different models on the same data with the same train/test split. Now we compare them side by side — not just on metrics, but visually. In a real business setting, you'd present this comparison table to stakeholders and recommend one model with justification.
</div>

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Side-by-side comparison table
# ═══════════════════════════════════════════════════════════════════════════════
comparison = pd.DataFrame({
    "Metric": ["RMSE (trips)", "R²", "Approach", "Parameters", "Handles Trend", "Requires Stationarity"],
    "SARIMAX": [
        f"{sarimax_rmse:,.0f}",
        f"{sarimax_r2:.4f}",
        "Statistical (ARIMA family)",
        f"({sarimax_params['p']},{sarimax_params['d']},{sarimax_params['q']})({sarimax_params['P']},{sarimax_params['D']},{sarimax_params['Q']},{sarimax_params['s']})",
        "Via differencing (d, D)",
        "Yes"
    ],
    "Prophet": [
        f"{prophet_rmse:,.0f}",
        f"{prophet_r2:.4f}",
        "Decomposable (trend + season)",
        "changepoint + seasonality priors",
        "Built-in trend component",
        "No"
    ]
})

print("╔══════════════════════════════════════════════╗")
print("║     SARIMAX  vs  Prophet — Comparison        ║")
print("╚══════════════════════════════════════════════╝\n")
print(comparison.to_string(index=False))

# Determine winner
if sarimax_rmse < prophet_rmse:
    print(f"\n🏆 SARIMAX wins on RMSE by {prophet_rmse - sarimax_rmse:,.0f} trips")
else:
    print(f"\n🏆 Prophet wins on RMSE by {sarimax_rmse - prophet_rmse:,.0f} trips")

### Combined Forecast Plot

In [ ]:
# Both forecasts on one chart for direct visual comparison
plt.figure(figsize=(14, 6))
plt.plot(train_data.index, train_data, label="Train", color="steelblue", alpha=0.7)
plt.plot(test_data.index, test_data, label="Test (Actual)", color="darkorange", linewidth=2.5)
plt.plot(test_data.index, sarimax_forecast, label=f"SARIMAX (RMSE={sarimax_rmse:,.0f})",
         linestyle="--", color="green", linewidth=2)
plt.plot(test_data.index, prophet_forecast, label=f"Prophet (RMSE={prophet_rmse:,.0f})",
         linestyle="--", color="purple", linewidth=2)

# Future forecasts
plt.plot(future_index, sarimax_future, linestyle=":", color="green", alpha=0.6, label="SARIMAX Future")
plt.plot(future_index, prophet_future, linestyle=":", color="purple", alpha=0.6, label="Prophet Future")

plt.axvline(x=test_data.index[0], color='red', linestyle='--', alpha=0.4)
plt.title("SARIMAX vs Prophet — Combined Forecast Comparison", fontsize=14)
plt.xlabel("Quarter")
plt.ylabel("Trips")
plt.legend(loc="upper left")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### Residual Analysis: Where Did Each Model Struggle?

In [ ]:
# Residuals = Actual - Predicted (positive = underforecast, negative = overforecast)
residuals = pd.DataFrame({
    "Quarter": test_data.index,
    "Actual": test_data.values,
    "SARIMAX_Forecast": sarimax_forecast.values,
    "Prophet_Forecast": prophet_forecast,
    "SARIMAX_Error": test_data.values - sarimax_forecast.values,
    "Prophet_Error": test_data.values - prophet_forecast
})

print("Residual Analysis — Test Period")
print("=" * 60)
print(residuals.to_string(index=False))

# Which quarters had the largest errors?
print(f"\nSARIMAX max error: Q{residuals.loc[residuals['SARIMAX_Error'].abs().idxmax(), 'Quarter'].quarter} "
      f"{residuals.loc[residuals['SARIMAX_Error'].abs().idxmax(), 'Quarter'].year} "
      f"({residuals['SARIMAX_Error'].abs().max():,.0f} trips)")
print(f"Prophet max error: Q{residuals.loc[residuals['Prophet_Error'].abs().idxmax(), 'Quarter'].quarter} "
      f"{residuals.loc[residuals['Prophet_Error'].abs().idxmax(), 'Quarter'].year} "
      f"({residuals['Prophet_Error'].abs().max():,.0f} trips)")

<div style="background-color: #FADBD8; border-left: 5px solid #E74C3C; padding: 15px; margin: 15px 0; border-radius: 4px;">
<strong style="color: #922B21;">🛑 STOP AND CHECK</strong><br>
<strong>Checkpoint — Section 7</strong><br><br>You should have:<br>• A comparison table with RMSE and R² for both models<br>• A combined forecast plot showing both models' predictions overlaid on actuals<br>• A residual table showing where each model was most wrong<br><br>There is no universally "better" model — the winner depends on the data. That's the whole point of comparing.
</div>

---
## Wrap-Up: What We Learned & What's Next

**Today's complete forecasting pipeline:**

| Step | What We Did | Key Function |
|------|------------|--------------|
| 1. Preprocess | Filter, aggregate, set datetime index | `preprocess_tourism_data()` |
| 2. EDA | Time plot, ADF stationarity test, seasonal decomposition | `adfuller()`, `seasonal_decompose()` |
| 3. Split | Temporal train/test (64/12 quarters) | `train_test_split_ts()` |
| 4. SARIMAX | auto_arima parameter search → fit → forecast → evaluate | `auto_arima()`, `SARIMAX()` |
| 5. Prophet | Direct params → fit → forecast → evaluate | `Prophet()` |
| 6. Compare | RMSE, R², combined plots, residual analysis | Side-by-side table |

**Key takeaways:**
- SARIMAX requires stationary data and explicit parameter tuning — but gives interpretable coefficients
- Prophet handles trend and seasonality automatically — but is a "black box" by comparison
- **Always compare multiple models** — the best approach depends on the specific dataset

**Next week (Week 3):** Regression — we move from forecasting *future values of the same variable* to predicting *one variable from others*. Linear → Multiple → Logistic regression builds the supervised learning foundation for Week 4's neural networks.

---
*CAP4767 Data Mining with Python | Miami Dade College | Spring 2026*